In [ ]:
import tensorflow as tf
import keras

from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from custom_metrics import f1, precision, recall
from keras.callbacks import TensorBoard

In [ ]:
config = tf.ConfigProto(device_count = {'GPU': 1 } ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
#custom parameters
nb_class = 225
hidden_dim = 128

resnet_model = VGGFace(model='resnet50',include_top=False, input_shape=(224, 224, 3))

In [ ]:
def resnet_input_preprocessor(x):
    return utils.preprocess_input(x , version=2)

In [ ]:
last_layer = resnet_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
# More layers can be added here
out = Dense(nb_class, activation='softmax', name='classifier')(x)
custom_resnet_model = Model(resnet_model.input, out)

In [ ]:
for layer in custom_resnet_model.layers[:-2]:
    layer.trainable = False

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=resnet_input_preprocessor)
validation_datagen = ImageDataGenerator(preprocessing_function=resnet_input_preprocessor)

# We can also use train_datagen.flow(X, label_X) but with flow_from_directory, the classname will be each directory
# Flow training images in batches of 64 using train_datagen generator

train_generator = train_datagen.flow_from_directory(
        './Data/train/',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical',
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        './Data/validation',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical',
        shuffle=False)

In [ ]:
sgd = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
custom_resnet_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy', f1, precision, recall])
NAME = 'Farzad-2Dense-255class-VGGFACE-RESNET50'
filepath= NAME + "-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,save_weights_only=False, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
callbacks_list = [checkpoint, tensorboard]

In [ ]:
custom_resnet_model.fit_generator(train_generator,
      steps_per_epoch=train_generator.samples//64,  
      epochs=20,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=validation_generator.samples//64,
      callbacks =callbacks_list
    )